In [39]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import sqlite3
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error


In [127]:
conn = sqlite3.connect("movie_info.db")
cur = conn.cursor() 
query = cur.execute(
    """
    SELECT m.Id,
        m.Title,
        g.Genre,
        m.Original_language,
        m.Overview ,
        m.Popularity ,
        m.Vote_average ,
        m.Vote_count ,
        m.Adult,
        m.Release_date,
        m.Backdrop_path ,
        m.Poster_path 
    FROM Movie m 
    LEFT JOIN Genre g ON g.GenreId = m.GenreId 
"""
) 

cols = [column[0] for column in query.description]
result = pd.DataFrame.from_records(data=query.fetchall(), columns=cols)
conn.close()

result

,Id,Title,Genre,Original_language,Overview,Popularity,Vote_average,Vote_count,Adult,Release_date,Backdrop_path,Poster_path
0,2,아리엘,드라마,fi,주인공 카스리넨은 광부다. 일하던 탄광이 폐광이 되며 도산을 하고 사장은 유일하게 ...,9.857,6.9,151,0,1988-10-21,/hQ4pYsIbP22TMXOUdSfC2mjWrO0.jpg,/ojDg0PGvs6R9xYFodRct2kdI6wC.jpg
1,3,천국의 그림자,드라마,fi,첫 장면부터 청소차가 새벽을 가른다. 청소부는 제각기 정해진 구역에서 익숙하게 쓰레...,9.119,7.2,149,0,1986-10-17,/l94l89eMmFKh7na2a1u5q67VgNx.jpg,/o2cWCX48flnEgDwSWu0Gpia0i0N.jpg
2,5,포룸,범죄,en,"새해 전날 밤, LA 몽시뇰 호텔에서 첫근무를 하게 된 괴짜 벨보이 테드는 상상조차...",14.118,5.7,2046,0,1995-12-09,/bCXLsO2MJWzYYBJoavGOZN5iqGn.jpg,/75aHn1NOYXh4M7L5shoeQ6NGykP.jpg
3,6,킬러 나이트,액션,en,어느날 프랭크(Frank Wyatt: 에밀리오 에스테베즈 분)와 마이크(Mike P...,10.615,6.5,215,0,1993-10-15,/5aXp2s4l6g5PcMMesIj63mx8hmJ.jpg,/rYFAvSPlQUCebayLcxyK79yvtvV.jpg
4,11,스타워즈: 에피소드 4 새로운 희망,모험,en,"공화국이 붕괴하고 제국이 수립된 뒤 20년, 제다이 기사단은 전멸하고 강력한 제국군...",62.500,8.2,16355,0,1977-05-25,/iPnH51khswDrYij6XIBHKlAznW.jpg,/7XFfURIFCJxN1mfBg0SAjk5yGzg.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...
29932,874244,Gingerbread Miracle,TV 영화,en,,6.216,6.3,6,0,2021-11-05,/u1polky3KHT3DhSMHYoypZa8p3I.jpg,/r6oT4u5nUNxKxpKVKbEhzX9pS8I.jpg
29933,876444,One December Night,TV 영화,en,,6.035,7.9,4,0,2021-11-13,/qHPd5T4huGl8FZqBNq7Ljk7lE0z.jpg,/jtzju1ZyZOEC6Vr29zqYaxfJAhr.jpg
29934,877004,"Next Stop, Christmas",TV 영화,en,,10.021,6.7,11,0,2021-11-06,/ohje04lAFnk7wozJ6mjb2wyMgWB.jpg,/f4Y8hRSAfGiSkGoKvS5sruW40me.jpg
29935,877690,Debbie Macomber's A Mrs. Miracle Christmas,TV 영화,en,,3.885,5.8,6,0,2021-11-06,/iFIv3Sz0dpasmzdSpO5qBK9hkmu.jpg,/rUrCxAs3byx0jnjcfWxjtGTQ0U.jpg


In [128]:
model = LinearRegression()

target = 'Popularity'

train, test = train_test_split(result,
                              test_size=0.8,  
                              random_state=2
                              )
                              
features = ['Vote_count', 'Vote_average']

X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]

# 모델 fit
model.fit(X_train, y_train)

# 트레이닝
y_pred = model.predict(X_train)
mae = mean_absolute_error(y_train, y_pred)
print(f'트레이닝 에러: {mae:.2f}')

# 테스트
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f'테스트 에러: {mae:.2f}')

트레이닝 에러: 5.01
테스트 에러: 6.09


In [129]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred)

print(f'MSE: {mse:.1f}, \nRMSE: {rmse:.1f}, \nMAE: {mae:.1f}, \nR2: {r2:.1f}')

MSE: 4305.9, 
RMSE: 65.6, 
MAE: 6.1, 
R2: 0.0


In [130]:
X_test

,Vote_count,Vote_average
24561,224,7.3
13934,54,6.1
1093,391,7.5
10365,46,6.1
9828,106,5.6
...,...,...
17753,4,5.5
9253,82,6.0
20426,4,5.8
21110,8,5.6


In [131]:
X_test['pop_pred'] = y_pred
X_test.reset_index(inplace=True)

<ipython-input-131-ccf0b635df1b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['pop_pred'] = y_pred


In [132]:
result.reset_index(inplace=True)

In [133]:
df = X_test.merge(result, on=['index', 'Vote_count', 'Vote_average'], how='left').drop(['index', 'Popularity'], axis=1)
df

,Vote_count,Vote_average,pop_pred,Id,Title,Genre,Original_language,Overview,Adult,Release_date,Backdrop_path,Poster_path
0,224,7.3,8.258189,258893,Scooby-Doo! WrestleMania Mystery,애니메이션,en,,0,2014-03-25,/opwf2L7m9MXGyFkiQndRDGdx7n2.jpg,/vgnxicD1qvhTrh4db9F7hJLoOaa.jpg
1,54,6.1,6.279323,47939,Three Wishes,판타지,en,,0,1995-10-27,/BuOA7HsDQmWLuvoTh7gCazSgCo.jpg,/5RjnorXfyCXPj6fnuXgdcFrb4gv.jpg
2,391,7.5,9.699259,1847,긴 이별,범죄,en,LA의 사설탐정 필립은 친구 테리의 부탁으로 그를 멕시코까지 차로 데려다 준다. 집...,0,1973-03-07,/9rm4TX4m6RHa5DbXDuRlVoawFU9.jpg,/oBhUK54yBJ0aH6u9zCzSV5iV7OP.jpg
3,46,6.1,6.215212,31016,사이드워크 오브 뉴욕,코미디,en,도시안에 8백만의 사람이 있다. 그중 완벽하게 맞는 2명이 만나기란... 앤과 3...,0,2001-11-23,/rMNIj68VWsZLTAyTkVsUXDLozG8.jpg,/AezUATLqaRj8gZUzpZOWQQsyTsI.jpg
4,106,5.6,6.439169,28503,Dementia 13,공포,en,,0,1963-09-25,/urDsmYHM1cduscLeev49s3vaVsW.jpg,/qSfelcriV7J7oOp24QKsry7tylY.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...
23945,4,5.5,5.570376,79107,La edad de piedra,코미디,es,,0,1964-12-24,None,/i3siquGLeEZwpJZqHiF2xrkEmhh.jpg
23946,82,6.0,6.452337,26480,레전드 오브 더 세븐 골든 뱀파이어스,액션,zh,,0,1974-07-11,/74QzCYsLihjSnWL6c2fm5CnMK80.jpg,/rzLw3KtszfsmkcjPQ5Ux7jz2igu.jpg
23947,4,5.8,5.724503,127949,King of the Cowboys,액션,en,,0,1943-04-09,/vn4PR4cLuC0F7lBu3Ym44m1Y1NI.jpg,/m0uaRhlUCe0viDsF4RN5V3MwQmI.jpg
23948,8,5.6,5.653807,148757,La casa del sorriso,드라마,it,,0,1991-03-01,/rjFsp7jU1VJ9rOIdh8xaQCYJEAA.jpg,/cBhYGT6qFEDZAtB8hEjPb1hzJAC.jpg


In [134]:
con = sqlite3.connect('database.db')
df.to_sql('info', con)